In [1]:
import pandas as pd
df = pd.read_csv("Resumo_v2.csv",sep=";")
df.head(5)

,RideID,Create,Updated,ProductID,Car,price,Tempo_Viagem_Min,Dia,Distância_km,Coordenadas_1,Coordenadas_2,H_Inicio,H_Fim
0,1425243,14/06/2022,14/06/2022,pop99,NaN,11.05,0.20,Terça-feira,2.03,"-22.9505577, -43.1826401","-22.9687019, -43.1852154",20:53:26.254623,20:53:38.244250
1,1425241,14/06/2022,14/06/2022,UberX,NaN,28.50,0.46,Terça-feira,5.13,"-23.5996561, -46.6655571","-23.5605827, -46.6925092",20:53:10.245554,20:53:38.011194
2,1425240,14/06/2022,14/06/2022,Comfort,NaN,22.00,0.14,Terça-feira,4.76,"-22.847022, -47.053341499999995","-22.8851178, -47.0319086",20:52:44.704490,20:52:53.343348
3,1425239,14/06/2022,14/06/2022,pop99,NaN,20.69,0.27,Terça-feira,4.64,"-25.4472447, -49.245239399999996","-25.4073641, -49.2593377",20:52:21.667225,20:52:37.892516
4,1425238,14/06/2022,14/06/2022,pop99,NaN,35.87,0.07,Terça-feira,4.91,"-23.5488482, -46.7149299","-23.5058317, -46.7265643",20:52:19.471241,20:52:23.596207


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score

# 🔹 Carregar os dados
df = pd.read_csv("Resumo_v2.csv", sep=";", na_values=["NaN", "nan"])

# 🔹 Remover linhas com preço ausente
df.dropna(subset=["price"], inplace=True)

# 🔹 Converter colunas categóricas
categorical_cols = ["ProductID", "Car", "Dia"]
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# 🔹 Criar features para tempo da viagem
df["Tempo_Viagem_Min"] = df["Tempo_Viagem_Min"].astype(float)

# 🔹 Criar features de localização (tratando erros na separação das coordenadas)
df["Lat1"], df["Lon1"] = zip(*df["Coordenadas_1"].apply(lambda x: x.split(",") if "," in x else ["NaN", "NaN"]))
df["Lat2"], df["Lon2"] = zip(*df["Coordenadas_2"].apply(lambda x: x.split(",") if "," in x else ["NaN", "NaN"]))

# 🔹 Converter coordenadas para float
df["Lat1"] = pd.to_numeric(df["Lat1"], errors='coerce')
df["Lon1"] = pd.to_numeric(df["Lon1"], errors='coerce')
df["Lat2"] = pd.to_numeric(df["Lat2"], errors='coerce')
df["Lon2"] = pd.to_numeric(df["Lon2"], errors='coerce')

# 🔹 Criar as diferenças entre latitude e longitude
df["Delta_Lat"] = df["Lat2"] - df["Lat1"]
df["Delta_Lon"] = df["Lon2"] - df["Lon1"]

# 🔹 Converter colunas de horário para formato numérico (minutos desde a meia-noite)
df["H_Inicio"] = pd.to_datetime(df["H_Inicio"], format="%H:%M:%S.%f", errors='coerce')
df["H_Fim"] = pd.to_datetime(df["H_Fim"], format="%H:%M:%S.%f", errors='coerce')

df["H_Inicio_Min"] = df["H_Inicio"].dt.hour * 60 + df["H_Inicio"].dt.minute
df["H_Fim_Min"] = df["H_Fim"].dt.hour * 60 + df["H_Fim"].dt.minute

# 🔹 Criar feature de duração da viagem
df["Duração_Viagem"] = df["H_Fim_Min"] - df["H_Inicio_Min"]

# 🔹 Selecionar features
features = ["ProductID", "Car", "Dia", "Tempo_Viagem_Min", "Distância_km", "Delta_Lat", "Delta_Lon",
            "H_Inicio_Min", "H_Fim_Min", "Duração_Viagem"]
X = df[features]
y = df["price"]

# 🔹 Remover possíveis valores NaN de forma segura
X = X.dropna().copy()
y = y.loc[X.index]  # Garantir alinhamento entre X e y

# 🔹 Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔹 Treinar modelo Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 🔹 Fazer previsões
y_pred = model.predict(X_test)

# 🔹 Avaliação do modelo
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 🔹 Exibir métricas
print(f"Erro médio absoluto (MAE): {mae:.2f}")
print(f"Acurácia (R² score): {r2:.2f}")

Erro médio absoluto (MAE): 4.83
Acurácia (R² score): 0.94


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import locale
import joblib # Importar joblib para salvar o modelo e encoders

# Configurar o locale para português para obter o nome do dia da semana correto
# Tentar configurar o locale. Mesmo que falhe, o mapeamento manual lidará com o nome do dia.
try:
    locale.setlocale(locale.LC_TIME, 'pt_BR.UTF8')
except locale.Error:
    print("Aviso: Não foi possível configurar o locale 'pt_BR.UTF8'. Nomes de dias da semana podem não estar em português.")
    try:
        locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil')
    except locale.Error:
        print("Aviso: Não foi possível configurar o locale 'Portuguese_Brazil'. Nomes de dias da semana podem não estar em português.")


# Função para mapear nomes de dias da semana para números (0-6)
def map_day_to_number(day_name):
    """Mapeia o nome de um dia da semana para um número (0 para Segunda, 6 para Domingo)."""
    # Mapeamento em inglês e português para maior robustez
    day_mapping = {
        'monday': 0, 'segunda-feira': 0, 'segunda': 0,
        'tuesday': 1, 'terça-feira': 1, 'terça': 1,
        'wednesday': 2, 'quarta-feira': 2, 'quarta': 2,
        'thursday': 3, 'quinta-feira': 3, 'quinta': 3,
        'friday': 4, 'sexta-feira': 4, 'sexta': 4,
        'saturday': 5, 'sábado': 5,
        'sunday': 6, 'domingo': 6
    }
    # Retorna o número mapeado ou None se o dia não for reconhecido
    return day_mapping.get(str(day_name).lower(), None)


# =============================================================================
# 🔹 CARREGAR E PRÉ-PROCESSAR OS DADOS, E TREINAR O MODELO
# =============================================================================

print("Carregando e pré-processando os dados, e treinando o modelo...")

# 🔹 Carregar os dados
try:
    df = pd.read_csv("Resumo_v2.csv", sep=";", na_values=["NaN", "nan"])
except FileNotFoundError:
    print("Erro: Arquivo 'Resumo_v2.csv' não encontrado. Certifique-se de que o arquivo está no mesmo diretório do script.")
    exit()

# 🔹 Remover linhas com preço ausente
df.dropna(subset=["price"], inplace=True)

# 🔹 Tratar a coluna 'Dia' primeiro: Mapear para números e remover a coluna original
print("Valores únicos na coluna 'Dia' antes do mapeamento:", df['Dia'].unique()) # Debugging
df['Dia_Numero'] = df['Dia'].apply(map_day_to_number)
# Remover a coluna 'Dia' original após mapear para evitar conflitos
df.drop('Dia', axis=1, inplace=True)
# Remover linhas onde o mapeamento do dia falhou (resultou em None)
df.dropna(subset=['Dia_Numero'], inplace=True)
# Converter para int (map_day_to_number retorna int ou None)
df['Dia_Numero'] = df['Dia_Numero'].astype(int)


# 🔹 Converter colunas categóricas (usando LabelEncoder apenas para ProductID)
categorical_cols_label_encoder = ["ProductID"]
label_encoders = {}
for col in categorical_cols_label_encoder:
    le = LabelEncoder()
    # Fit o encoder nos dados originais para aprender todas as categorias possíveis
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


# 🔹 Criar features para tempo da viagem
df["Tempo_Viagem_Min"] = df["Tempo_Viagem_Min"].astype(float)

# 🔹 Criar features de localização (tratando erros na separação das coordenadas)
# Preservar colunas originais caso sejam necessárias para depuração
df['Coordenadas_1_split'] = df["Coordenadas_1"].apply(lambda x: x.split(",") if isinstance(x, str) and "," in x else ["NaN", "NaN"])
df['Coordenadas_2_split'] = df["Coordenadas_2"].apply(lambda x: x.split(",") if isinstance(x, str) and "," in x else ["NaN", "NaN"])

df["Lat1"] = df['Coordenadas_1_split'].apply(lambda x: x[0])
df["Lon1"] = df['Coordenadas_1_split'].apply(lambda x: x[1])
df["Lat2"] = df['Coordenadas_2_split'].apply(lambda x: x[0])
df["Lon2"] = df['Coordenadas_2_split'].apply(lambda x: x[1])

# 🔹 Converter coordenadas para float
df["Lat1"] = pd.to_numeric(df["Lat1"], errors='coerce')
df["Lon1"] = pd.to_numeric(df["Lon1"], errors='coerce')
df["Lat2"] = pd.to_numeric(df["Lat2"], errors='coerce')
df["Lon2"] = pd.to_numeric(df["Lon2"], errors='coerce')

# 🔹 Criar as diferenças entre latitude e longitude
df["Delta_Lat"] = df["Lat2"] - df["Lat1"]
df["Delta_Lon"] = df["Lon2"] - df["Lon1"]

# 🔹 Converter colunas de horário para formato numérico (minutos desde a meia-noite)
# Tratar horários que podem não estar no formato esperado
df["H_Inicio_dt"] = pd.to_datetime(df["H_Inicio"], format="%H:%M:%S.%f", errors='coerce')
df["H_Fim_dt"] = pd.to_datetime(df["H_Fim"], format="%H:%M:%S.%f", errors='coerce')

# Calcular minutos desde a meia-noite, tratando NaT
df["H_Inicio_Min"] = np.nan
valid_inicio_mask = df["H_Inicio_dt"].notna()
df.loc[valid_inicio_mask, "H_Inicio_Min"] = df.loc[valid_inicio_mask, "H_Inicio_dt"].dt.hour * 60 + df.loc[valid_inicio_mask, "H_Inicio_dt"].dt.minute

df["H_Fim_Min"] = np.nan
valid_fim_mask = df["H_Fim_dt"].notna()
df.loc[valid_fim_mask, "H_Fim_Min"] = df.loc[valid_fim_mask, "H_Fim_dt"].dt.hour * 60 + df.loc[valid_fim_mask, "H_Fim_dt"].dt.minute

# 🔹 Criar feature de duração da viagem
df["Duração_Viagem"] = df["H_Fim_Min"] - df["H_Inicio_Min"]

# 🔹 Selecionar features
# Usar 'Dia_Numero' em vez de 'Dia'
features = ["ProductID", "Dia_Numero", "Tempo_Viagem_Min", "Distância_km", "Delta_Lat", "Delta_Lon",
            "H_Inicio_Min", "H_Fim_Min", "Duração_Viagem"]
X = df[features]
y = df["price"]

# 🔹 Remover possíveis valores NaN de forma segura
# Remover linhas onde qualquer uma das features selecionadas ou o target é NaN
# Usar apenas as features selecionadas para o dropna
df_cleaned = df.dropna(subset=features + ["price"]).copy()
X_cleaned = df_cleaned[features]
y_cleaned = df_cleaned["price"]

# 🔹 Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# 🔹 Treinar modelo Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("Modelo treinado com sucesso!")

# =============================================================================
# 🔹 SALVAR MODELO E ENCODERS
# =============================================================================
MODEL_FILENAME = "modelo_treinado.joblib"
ENCODERS_FILENAME = "encoders.joblib"
FEATURES_FILENAME = "features.joblib" # Salvar também a lista de features

print(f"Salvando modelo, encoders e lista de features em '{MODEL_FILENAME}', '{ENCODERS_FILENAME}' e '{FEATURES_FILENAME}'...")

try:
    joblib.dump(model, MODEL_FILENAME)
    joblib.dump(label_encoders, ENCODERS_FILENAME)
    joblib.dump(features, FEATURES_FILENAME)
    print("Arquivos salvos com sucesso!")
except Exception as e:
    print(f"Erro ao salvar arquivos: {e}")


Aviso: Não foi possível configurar o locale 'pt_BR.UTF8'. Nomes de dias da semana podem não estar em português.
Aviso: Não foi possível configurar o locale 'Portuguese_Brazil'. Nomes de dias da semana podem não estar em português.
Carregando e pré-processando os dados, e treinando o modelo...
Valores únicos na coluna 'Dia' antes do mapeamento: ['Terça-feira' 'Segunda-feira' 'Domingo' 'Sábado' 'Sexta-feira'
 'Quinta-feira' 'Quarta-feira']
Modelo treinado com sucesso!
Salvando modelo, encoders e lista de features em 'modelo_treinado.joblib', 'encoders.joblib' e 'features.joblib'...
Arquivos salvos com sucesso!


In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
import locale
from geopy.geocoders import Nominatim
import requests
import joblib
from sklearn.metrics import mean_absolute_error, r2_score

# Configuração inicial
try:
    locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')
except locale.Error:
    try:
        # Tentativa alternativa para Windows
        locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil')
    except locale.Error:
        # Fallback que funciona na maioria dos sistemas
        print("⚠️ Usando locale 'pt_BR' como fallback. Nomes de dias podem estar em inglês.")
geolocator = Nominatim(user_agent="previsao_viagem")

# =============================================================================
# 🔹 Configuração da API de Roteamento HERE
# =============================================================================
HERE_API_KEY = "zK9f9fu8wVtDG9dCY4fUvfpO6K4LDdRYtQupwo89fQ4"
HERE_ROUTING_URL = "https://router.hereapi.com/v8/routes"

# =============================================================================
# 🔹 Funções Auxiliares
# =============================================================================
def get_coordinates(address):
    """Converte um endereço em coordenadas (latitude, longitude)."""
    try:
        location = geolocator.geocode(address)
        return (location.latitude, location.longitude) if location else None
    except Exception as e:
        print(f"Erro ao geocodificar '{address}': {e}")
        return None

def get_route_data(origin_coords, destination_coords, api_key):
    """Obtém distância e duração da rota usando API HERE."""
    params = {
        "transportMode": "car",
        "origin": f"{origin_coords[0]},{origin_coords[1]}",
        "destination": f"{destination_coords[0]},{destination_coords[1]}",
        "return": "summary",
        "apikey": api_key
    }

    try:
        response = requests.get(HERE_ROUTING_URL, params=params)
        data = response.json()

        if data.get("routes"):
            section = data["routes"][0]["sections"][0]
            summary = section["summary"]
            return (summary["length"]/1000, summary["duration"]/60)  # km, minutos
        return (None, None)
    except Exception as e:
        print(f"Erro na API HERE: {e}")
        return (None, None)

def map_day_to_number(day_name):
    """Mapeia nome do dia para número (0=segunda)."""
    day_mapping = {
        'segunda-feira': 0, 'terça-feira': 1, 'quarta-feira': 2,
        'quinta-feira': 3, 'sexta-feira': 4, 'sábado': 5, 'domingo': 6
    }
    return day_mapping.get(day_name.lower().split('-')[0], 0)

# =============================================================================
# 🔹 Carregar Modelo e Encoders
# =============================================================================
try:
    model = joblib.load("modelo_treinado.joblib")
    label_encoders = joblib.load("encoders.joblib")
    features = joblib.load("features.joblib")
    print("✔ Modelo e encoders carregados")
except Exception as e:
    print(f"Erro ao carregar arquivos: {e}")
    exit()

# =============================================================================
# 🔹 Interação com o Usuário
# =============================================================================
print("\n" + "="*40)
print(" PREVISÃO DE PREÇO DE VIAGEM ")
print("="*40)

# 🔹 Categorias disponíveis (filtradas)
CATEGORIAS_DISPONIVEIS = ["pop99", "UberX", "Comfort", "Taxi"]

# Obter dados do usuário
try:
    endereco_origem = input("\nDigite o endereço de origem (ex: Av. Paulista, 1000, São Paulo): ").strip()
    endereco_destino = input("Digite o endereço de destino (ex: Parque Ibirapuera, São Paulo): ").strip()

    print("\nCategorias disponíveis:")
    for i, cat in enumerate(CATEGORIAS_DISPONIVEIS, 1):
        print(f"{i}. {cat}")

    escolha = int(input("Escolha o número da categoria: ")) - 1
    categoria_escolhida = CATEGORIAS_DISPONIVEIS[escolha]
except Exception as e:
    print(f"Erro na entrada: {e}")
    exit()

# =============================================================================
# 🔹 Processamento dos Dados
# =============================================================================
# Geocodificação
print("\n📍 Obtendo coordenadas...")
coords_origem = get_coordinates(endereco_origem)
coords_destino = get_coordinates(endereco_destino)

if not coords_origem or not coords_destino:
    print("Erro ao geocodificar endereços")
    exit()

# Obter dados da rota
print("🛣️ Calculando rota...")
distancia_km, duracao_min = get_route_data(coords_origem, coords_destino, HERE_API_KEY)

if not distancia_km or not duracao_min:
    print("Erro ao calcular rota")
    exit()

# Verificação de plausibilidade
print("\n" + "="*40)
print(" VERIFICAÇÃO DOS DADOS ")
print("="*40)
print(f"Distância calculada: {distancia_km:.2f} km")
print(f"Duração estimada: {duracao_min:.1f} minutos")

if distancia_km > 50:
    print("\n⚠️ AVISO: Distância acima do normal para trajeto urbano")
    print("Verifique se os endereços estão corretos")

# Preparar dados para previsão
now = datetime.now()
dados_viagem = {
    "ProductID": [categoria_escolhida],
    "Dia_Numero": [map_day_to_number(now.strftime("%A"))],
    "Tempo_Viagem_Min": [duracao_min],
    "Distância_km": [distancia_km],
    "Delta_Lat": [coords_destino[0] - coords_origem[0]],
    "Delta_Lon": [coords_destino[1] - coords_origem[1]],
    "H_Inicio_Min": [now.hour * 60 + now.minute],
    "H_Fim_Min": [(now.hour * 60 + now.minute) + duracao_min],
    "Duração_Viagem": [duracao_min]
}

# Codificar categoria
try:
    dados_viagem["ProductID"] = label_encoders["ProductID"].transform([categoria_escolhida])
except ValueError:
    print(f"Erro: Categoria '{categoria_escolhida}' não reconhecida pelo modelo")
    exit()

# =============================================================================
# 🔹 Previsão e Comparação
# =============================================================================
def prever_preco(categoria):
    """Faz previsão para uma categoria específica."""
    dados = dados_viagem.copy()
    try:
        dados["ProductID"] = label_encoders["ProductID"].transform([categoria])
        return model.predict(pd.DataFrame(dados)[features])[0]
    except:
        return None

# Prever para todas as categorias
print("\n🔍 Calculando preços...")
precos = {}
for cat in CATEGORIAS_DISPONIVEIS:
    preco = prever_preco(cat)
    if preco is not None:
        precos[cat] = preco

# Encontrar a mais barata
if precos:
    categoria_mais_barata = min(precos, key=precos.get)

    # Resultado principal
    print("\n" + "="*40)
    print(f" {categoria_escolhida.upper()}: R$ {precos[categoria_escolhida]:.2f} ")
    print("="*40)

    # Comparação
    print("\n💲 COMPARAÇÃO DE PREÇOS")
    for cat, valor in sorted(precos.items(), key=lambda x: x[1]):
        print(f"{cat.upper():<10}: R$ {valor:.2f}")

    if categoria_mais_barata != categoria_escolhida:
        economia = precos[categoria_escolhida] - precos[categoria_mais_barata]
        print(f"\n💡 Você economizaria R$ {economia:.2f} com {categoria_mais_barata.upper()}")
else:
    print("Erro: Não foi possível calcular preços")

print("="*40)

⚠️ Usando locale 'pt_BR' como fallback. Nomes de dias podem estar em inglês.
✔ Modelo e encoders carregados

 PREVISÃO DE PREÇO DE VIAGEM 

Digite o endereço de origem (ex: Av. Paulista, 1000, São Paulo): Av. Paulista, 1000, São Paulo
Digite o endereço de destino (ex: Parque Ibirapuera, São Paulo): Av. Liberdade, 500, São Paulo

Categorias disponíveis:
1. pop99
2. UberX
3. Comfort
4. Taxi
Escolha o número da categoria: 1

📍 Obtendo coordenadas...


🛣️ Calculando rota...

 VERIFICAÇÃO DOS DADOS 
Distância calculada: 44.13 km
Duração estimada: 59.4 minutos

🔍 Calculando preços...

 POP99: R$ 183.34 

💲 COMPARAÇÃO DE PREÇOS
TAXI      : R$ 99.75
UBERX     : R$ 107.67
COMFORT   : R$ 147.59
POP99     : R$ 183.34

💡 Você economizaria R$ 83.58 com TAXI
